<a href="https://colab.research.google.com/github/ra4ola/APA-Assigment2/blob/main/APA2023_A3_Part2_DQN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Assignment 3 - Part 2



#Dependencias

In [1]:
!pip install gymnasium

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 953.9/953.9 kB 12.4 MB/s eta 0:00:00


# Robot Enviroment

In [2]:
import gymnasium  as gym
from gymnasium import spaces
import numpy as np
import matplotlib.pyplot as plt
import math
from IPython import display


from enum import Enum

class Modes(Enum):
    PoseOnly = 1
    PoseAndLocalMap = 2
    GlobalMap = 3
    PoseAndLocalMapInline = 4


class APA2023SimpleRobotEnv(gym.Env):

    metadata = {"render_modes": ["human", "rgb_array"], "render_fps": 60}

    def __init__(self, grid_size,local_grid_size,mode=Modes.PoseOnly):
        super(APA2023SimpleRobotEnv, self).__init__()


        self.grid_size=grid_size
        self.local_grid_size=local_grid_size
        # Define the action space (up, down, left, right)
        self.action_space = spaces.Discrete(3)

        self.mode=mode
        self.TotalReward=0;

        # Define the observation space (grid world with discrete positions)
        self.observation_space = spaces.Discrete(grid_size)
        self.gridmap = np.zeros((self.observation_space.n, self.observation_space.n), dtype = np.float64)
        self.heatmap = np.zeros((self.observation_space.n, self.observation_space.n), dtype = np.int32)

        self.localgridmap = np.zeros((local_grid_size, local_grid_size), dtype = np.float64)

        for i in range(self.observation_space.n):
            self.gridmap[i][0]=1
            self.gridmap[0][i]=1
            self.gridmap[i][self.observation_space.n-1]=1
            self.gridmap[self.observation_space.n-1][i]=1

        # Set the initial state of the robot
        self.robot_position = np.array([0, 0,0])
        self.reset_robot_position= np.array([0, 0,0])

        # Set the goal position
        self.goal_position = np.array([1, 1])
        self.path=[self.robot_position]

        # Set the maximum number of steps
        self.max_steps = 1000

        # Initialize step counter
        self.current_step = 0
        self.gridResolution=0.25;
        self._last_action=-1;
        self._score=0;
        self.render_mode="human"
        self.lastRGB=None
        self.background=None


    @property
    def info(self):
        return {
            "goal": {
                "x": self.goal_position[0],
                "y": self.goal_position[1]
            },
            "robot": {
                "x": self.robot_position[0],
                "y": self.robot_position[1],
                "theta": self.robot_position[2]
            },
            "last_action": self._last_action,
            "score": self._score
        }
    def addObstacle(self,obstacle):
        # Reset the environment to initial state
        x=obstacle[0]/self.gridResolution+self.observation_space.n/2.0
        y=obstacle[1]/self.gridResolution+self.observation_space.n/2.0


        # simple collision checking
        self.gridmap[int(y)][int(x)]=1;

    def addObstacleLine(self, start, end, value):
        """
        Draw a line with the specified value in the environment grid using Bresenham's algorithm.

        Parameters:
            start (tuple): Starting position (x, y).
            end (tuple): Ending position (x, y).
            value (float): Value to fill the line with.

        Returns:
            None
        """

        y0=int(start[1]/self.gridResolution+self.observation_space.n/2.0)
        x0=int(start[0]/self.gridResolution+self.observation_space.n/2.0)
        y1=int(end[1]/self.gridResolution+self.observation_space.n/2.0)
        x1=int(end[0]/self.gridResolution+self.observation_space.n/2.0)


        dx = x1 - x0
        dy = y1 - y0

        xsign = 1 if dx > 0 else -1
        ysign = 1 if dy > 0 else -1

        dx = abs(dx)
        dy = abs(dy)

        if dx > dy:
            xx, xy, yx, yy = xsign, 0, 0, ysign
        else:
            dx, dy = dy, dx
            xx, xy, yx, yy = 0, ysign, xsign, 0

        D = 2*dy - dx
        y = 0

        for x in range(dx + 1):
            self.gridmap[int(y0 + x*xy + y*yy)][int(x0 + x*xx + y*yx)] = value
            if D >= 0:
                y += 1
                D -= 2*dx
            D += 2*dy

    def removeObstacle(self,obstacle):
        # Reset the environment to initial state
        x=obstacle[0]/self.gridResolution+self.observation_space.n/2.0
        y=obstacle[1]/self.gridResolution+self.observation_space.n/2.0


        # simple collision checking
        self.gridmap[int(y)][int(x)]=1;



    def reset(self,seed=-1,options=None):
        # Reset the environment to initial state
        self.robot_position = self.reset_robot_position
        self.current_step = 0
        self.path=[self.robot_position]
        self._last_action=-1
        self._score=0
        self.TotalReward=0
        self.lastdist=np.linalg.norm(self.robot_position[:2] - self.goal_position)

        self.localgridmap = np.zeros((self.local_grid_size, self.local_grid_size), dtype = np.float64)

        if(self.mode==Modes.PoseOnly):
           out=self.robot_position.copy()
           out[0]=self.goal_position[0]-self.robot_position[0]
           out[1]=self.goal_position[1]-self.robot_position[1]
           out[0]=out[0]/math.sqrt(out[0]*out[0]+out[1]*out[1])
           out[1]=out[1]/math.sqrt(out[0]*out[0]+out[1]*out[1])


           return out, self.info

        elif(self.mode==Modes.PoseAndLocalMap):
           out=self.robot_position.copy()
           out[0]=self.goal_position[0]-self.robot_position[0]
           out[1]=self.goal_position[1]-self.robot_position[1]
           out[0]=out[0]/math.sqrt(out[0]*out[0]+out[1]*out[1])
           out[1]=out[1]/math.sqrt(out[0]*out[0]+out[1]*out[1])
           return (out,self.localgridmap), self.info

        elif(self.mode==Modes.PoseAndLocalMapInline):
           out=self.robot_position.copy()
           out[0]=self.goal_position[0]-self.robot_position[0]
           out[1]=self.goal_position[1]-self.robot_position[1]
           out[0]=out[0]/math.sqrt(out[0]*out[0]+out[1]*out[1])
           out[1]=out[1]/math.sqrt(out[0]*out[0]+out[1]*out[1])
           out=np.concatenate((out, self.localgridmap.flatten()), axis=0)
           return out, self.info

        elif(self.mode==Modes.GlobalMap):
           x=self.robot_position[0]/self.gridResolution+self.observation_space.n/2.0
           y=self.robot_position[1]/self.gridResolution+self.observation_space.n/2.0
           globalmap=self.gridmap.copy()
           return (self.robot_position,globalmap), self.info

        return self.robot_position, self.info


    def setGoal(self,goal):
        # Set robot's goal
        self.goal_position=goal
        self.lastdist=np.linalg.norm(self.robot_position[:2] - self.goal_position)

    def setPosition(self,position):
        # Set robot's initial state
        self.robot_position =position
        self.reset_robot_position =position
        self.path=[self.robot_position]



    def step(self, action):

        self._last_action=action

        if (action<0):  # to generate still frames
            return self.robot_position, 0, False, False, self.info

        # Define the action mapping (0: up, 1: down, 2: left, 3: right)
        action_mapping = {
            0: np.array([0.1, 0]),
            1: np.array([0.1, 0.5]),
            2: np.array([0.1, -0.5]),
        }

        h=1.0;
        # Compute new pose
        x = self.robot_position[0] + h*action_mapping[action][0]*math.cos(self.robot_position[2])
        y = self.robot_position[1] + h*action_mapping[action][0]*math.sin(self.robot_position[2])
        theta = self.robot_position[2] + h*action_mapping[action][1]
        theta= math.atan2(math.sin(theta), math.cos(theta))

        collision=0

        # Check if maximum steps reached
        done = self.current_step >= self.max_steps  # defined in this class



        # Check if the new position is valid
        if (-self.gridResolution*self.observation_space.n/2 <= x < self.gridResolution*self.observation_space.n/2) and (-self.gridResolution*self.observation_space.n/2 <= y < self.gridResolution*self.observation_space.n/2):
            self.robot_position = np.array([x, y,theta])
        else:
          #if it is here then it collided with obstacles in the environment
          done=True
          collision=-1000
        x=self.robot_position[0]/self.gridResolution+self.observation_space.n/2.0
        y=self.robot_position[1]/self.gridResolution+self.observation_space.n/2.0

        self.heatmap[int(y)][int(x)]=self.heatmap[int(y)][int(x)]+1

        # simple collision checking
        if self.gridmap[int(y)][int(x)]>0:
          done=True
          collision=-1000

        # Calculate the reward (negative distance to goal)
        d=0.0
        dx=np.linalg.norm(self.robot_position[:2] - self.goal_position)
        delta=self.lastdist-dx
        self.lastdist=dx

        if(dx<0.25 and not done):
          d=100;
          done=True
          print('reached goal')

        reward = -0.1*dx+d+collision-self.current_step*0.01   #+0.1*delta
        self.TotalReward =self.TotalReward+reward;

        # Increment step counter
        self.current_step += 1


        self.path.append(self.robot_position)
        self.localgridmap=np.zeros((self.local_grid_size, self.local_grid_size), dtype = np.float64)

        x=self.robot_position[0]/self.gridResolution+self.observation_space.n/2.0
        y=self.robot_position[1]/self.gridResolution+self.observation_space.n/2.0

        xg=int(self.goal_position[0]/self.gridResolution+self.observation_space.n/2.0)
        yg=int(self.goal_position[1]/self.gridResolution+self.observation_space.n/2.0)

        for i in range(self.local_grid_size):
          for j in range(self.local_grid_size):
              if(i==xg and j==yg and (0 <= int(x+i-self.local_grid_size/2)<self.observation_space.n and 0 <= int(y+j-self.local_grid_size/2)<self.observation_space.n)):
                 self.localgridmap[j][i]= -1

              elif 0 <= int(x+i-self.local_grid_size/2)<self.observation_space.n and 0 <= int(y+j-self.local_grid_size/2)<self.observation_space.n:
                 self.localgridmap[j][i]= self.gridmap[int(y+j-self.local_grid_size/2)][int(x+i-self.local_grid_size/2)]

        Truncated=done

        if(self.mode==Modes.PoseOnly):
           out=self.robot_position.copy()
           out[0]=self.goal_position[0]-self.robot_position[0]
           out[1]=self.goal_position[1]-self.robot_position[1]
           out[0]=out[0]/math.sqrt(out[0]*out[0]+out[1]*out[1])
           out[1]=out[1]/math.sqrt(out[0]*out[0]+out[1]*out[1])
           return out , reward, done, Truncated, self.info

        elif(self.mode==Modes.PoseAndLocalMap):
           return (self.robot_position,self.localgridmap), reward, done, Truncated, self.info

        elif(self.mode==Modes.PoseAndLocalMapInline):
           out=self.robot_position.copy()
           out[0]=self.goal_position[0]-self.robot_position[0]
           out[1]=self.goal_position[1]-self.robot_position[1]
           out[0]=out[0]/math.sqrt(out[0]*out[0]+out[1]*out[1])
           out[1]=out[1]/math.sqrt(out[0]*out[0]+out[1]*out[1])
           out=np.concatenate((out, self.localgridmap.flatten()), axis=0)
           return out, reward, done, Truncated, self.info

        elif(self.mode==Modes.GlobalMap):
           globalmap=self.gridmap.copy()
           return (self.robot_position,globalmap), reward, done, Truncated, self.info

        return self.robot_position, reward, done, Truncated, self.info

    def render(self):

        if self.render_mode == "human":
          figure, (axes, axes2) = plt.subplots(1, 2)

          axes.axis([-self.gridResolution*self.observation_space.n/2,self.gridResolution*self.observation_space.n/2,-self.gridResolution*self.observation_space.n/2,self.gridResolution*self.observation_space.n/2])
          axes.axis('square')
          axes2.axis([-self.gridResolution*self.local_grid_size/2,self.gridResolution*self.local_grid_size/2,-self.gridResolution*self.local_grid_size/2,self.gridResolution*self.local_grid_size/2])
          axes2.axis('square')


          for i in range(self.local_grid_size):
              for j in range(self.local_grid_size):
                  if(self.localgridmap[i][j]>0):
                    axes2.add_artist( plt.Rectangle(((j-self.local_grid_size/2)*self.gridResolution, (i-self.local_grid_size/2)*self.gridResolution), self.gridResolution, self.gridResolution, fill=True, color='grey', linewidth=2))
                  #else:
                  #  axes2.add_artist( plt.Rectangle(((j-self.local_grid_size/2)*self.gridResolution, (i-self.local_grid_size/2)*self.gridResolution), self.gridResolution, self.gridResolution, fill=False, color='black', linewidth=0.1))



          for i in range(self.observation_space.n):
              for j in range(self.observation_space.n):
                  if(self.gridmap[i][j]>0):
                    axes.add_artist( plt.Rectangle(((j-self.observation_space.n/2)*self.gridResolution, (i-self.observation_space.n/2)*self.gridResolution), self.gridResolution, self.gridResolution, fill=True, color='grey', linewidth=2))
                  #else:
                  #  axes.add_artist( plt.Rectangle(((j-self.observation_space.n/2)*self.gridResolution, (i-self.observation_space.n/2)*self.gridResolution), self.gridResolution, self.gridResolution, fill=False, color='black', linewidth=0.1))

          if(len(self.path))>1:
              for i in range(len(self.path)-1):
                  axes.plot([self.path[i][0],self.path[i+1][0]], [self.path[i][1],self.path[i+1][1]], color='violet', linewidth=1)



          # Display the robot and goal
          axes.add_artist(plt.Circle((self.robot_position[0], self.robot_position[1]), 0.1, color='blue'))
          axes.add_artist(plt.Circle((self.goal_position[0], self.goal_position[1]), 0.1, color='green'))
          x=[self.robot_position[0],self.robot_position[0]+0.2*math.cos(self.robot_position[2])]
          y=[self.robot_position[1],self.robot_position[1]+0.2*math.sin(self.robot_position[2])]


          axes.plot(x, y, color='red', linewidth=1)

          axes2.add_artist(plt.Circle((0, 0), 0.1, color='blue'))

          # Represent robot yaw
          x=[0,0+0.2*math.cos(self.robot_position[2])]
          y=[0,0+0.2*math.sin(self.robot_position[2])]
          axes2.plot(x, y, color='red', linewidth=1)

          plt.xticks(fontsize=10)
          plt.text(-1.25, -1.55, r'Total Reward: '+str(self.TotalReward), fontsize=10)

          axes2.tick_params(left = False, right = False , labelleft = False , labelbottom = False, bottom = False)

          display.clear_output(wait=True)
          plt.show()
        if self.render_mode == "rgb_array":

          if(self._last_action==-1 and self.lastRGB is not None):
            return self.lastRGB;

          plt.switch_backend('agg')
          figure, (axes, axes2) = plt.subplots(1, 2)
          canvas = figure.canvas

          axes.axis([-self.gridResolution*self.observation_space.n/2,self.gridResolution*self.observation_space.n/2,-self.gridResolution*self.observation_space.n/2,self.gridResolution*self.observation_space.n/2])
          axes.axis('square')

          axes2.axis([-self.gridResolution*self.local_grid_size/2,self.gridResolution*self.local_grid_size/2,-self.gridResolution*self.local_grid_size/2,self.gridResolution*self.local_grid_size/2])
          axes2.axis('square')
          drawback=1

          if self.background is None:

            for i in range(self.local_grid_size):
                for j in range(self.local_grid_size):
                    if(self.localgridmap[i][j]>0):
                      axes2.add_artist( plt.Rectangle(((j-self.local_grid_size/2)*self.gridResolution, (i-self.local_grid_size/2)*self.gridResolution), self.gridResolution, self.gridResolution, fill=True, color='grey', linewidth=2))
                    #else:
                      #axes2.add_artist( plt.Rectangle(((j-self.local_grid_size/2)*self.gridResolution, (i-self.local_grid_size/2)*self.gridResolution), self.gridResolution, self.gridResolution, fill=False, color='black', linewidth=0.1))



            for i in range(self.observation_space.n):
                for j in range(self.observation_space.n):
                    if(self.gridmap[i][j]>0):
                      axes.add_artist( plt.Rectangle(((j-self.observation_space.n/2)*self.gridResolution, (i-self.observation_space.n/2)*self.gridResolution), self.gridResolution, self.gridResolution, fill=True, color='grey', linewidth=2))
                    #else:
                    #  axes.add_artist( plt.Rectangle(((j-self.observation_space.n/2)*self.gridResolution, (i-self.observation_space.n/2)*self.gridResolution), self.gridResolution, self.gridResolution, fill=False, color='black', linewidth=0.1))

            axes2.tick_params(left = False, right = False , labelleft = False ,labelbottom = False, bottom = False)


          if(len(self.path))>1:
            for i in range(len(self.path)-1):
                axes.plot([self.path[i][0],self.path[i+1][0]], [self.path[i][1],self.path[i+1][1]], color='violet', linewidth=1)



          # Display the grid world
          axes.add_artist(plt.Circle((self.robot_position[0], self.robot_position[1]), 0.1, color='blue'))
          axes.add_artist(plt.Circle((self.goal_position[0], self.goal_position[1]), 0.1, color='green'))
          x=[self.robot_position[0],self.robot_position[0]+0.2*math.cos(self.robot_position[2])]
          y=[self.robot_position[1],self.robot_position[1]+0.2*math.sin(self.robot_position[2])]


          axes.plot(x, y, color='red', linewidth=1)
          axes2.add_artist(plt.Circle((0, 0), 0.1, color='blue'))

          x=[0,0+0.2*math.cos(self.robot_position[2])]
          y=[0,0+0.2*math.sin(self.robot_position[2])]
          axes2.plot(x, y, color='red', linewidth=1)

          #plt.title("[APA2023] Simple Robot Environment")
          plt.xticks(fontsize=10)
          plt.text(-1.25, -1.55, r'Total Reward: '+str(self.TotalReward), fontsize=10)
          axes2.tick_params(left = False, right = False , labelleft = False ,labelbottom = False, bottom = False)


          canvas.draw();
          image_flat = np.frombuffer(canvas.tostring_rgb(), dtype='uint8')  # (H * W * 3,)
          image = image_flat.reshape(*reversed(canvas.get_width_height()), 3)  # (H, W, 3)
          plt.close()
          self.lastRGB=image;
          return image;




In [3]:
# Initial scenario -> Empty environment (border-only obstacles)
env = APA2023SimpleRobotEnv(grid_size=50,local_grid_size=11)
observation = env.reset()
env.setGoal(np.array([5, 5]))
env.setPosition(np.array([-2,-2.5,0 ]))

# Deep RL Replay Buffer

In [4]:
class ReplayBuffer:
    def __init__(self, size):
        self.size = size
        self.counter = 0
        self.state_buffer = []
        self.action_buffer = []
        self.reward_buffer = []
        self.new_state_buffer = []
        self.terminal_buffer = []

    def store_tuples(self, state, action, reward, new_state, done):
        if len(self.state_buffer) < self.size:
            self.state_buffer.append(None)
            self.action_buffer.append(None)
            self.reward_buffer.append(None)
            self.new_state_buffer.append(None)
            self.terminal_buffer.append(None)

        self.counter = self.counter % self.size
        self.state_buffer[self.counter] = state
        self.action_buffer[self.counter] = action
        self.reward_buffer[self.counter] = reward
        self.new_state_buffer[self.counter] = new_state
        self.terminal_buffer[self.counter] = done

        #reward propagation (collision only)
        if(reward.item()<-999.0):
            self.reward_buffer[self.counter-1]=self.reward_buffer[self.counter-1]+reward
            self.reward_buffer[self.counter-2]=self.reward_buffer[self.counter-2]+reward
            self.reward_buffer[self.counter-3]=self.reward_buffer[self.counter-3]+reward
            self.reward_buffer[self.counter-4]=self.reward_buffer[self.counter-4]+reward

        self.counter += 1

    def sample_buffer(self, batch_size):
        max_buffer = min(self.counter, self.size)
        batch = np.random.choice(max_buffer, batch_size, replace=False)
        state_batch=[];
        action_batch=[];
        reward_batch=[];
        new_state_batch=[];
        done_batch=[];

        for a in batch:
            state_batch.append(self.state_buffer[a])
            action_batch.append(self.action_buffer[a])
            reward_batch.append(self.reward_buffer[a])
            new_state_batch.append(self.new_state_buffer[a])
            done_batch.append(self.terminal_buffer[a])
        return state_batch, action_batch, reward_batch, new_state_batch, done_batch





# DQN arquitecture

In [5]:
from torchvision import datasets, models
import torch.nn as nn
import torch
from torch.utils import data
import torch.nn as nn
import torch.optim as optim
import torch.autograd as autograd
import random

import glob
import io
import base64
from IPython.display import HTML
from IPython import display as ipythondisplay

device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')


class DQN(nn.Module):

    def __init__(self, inputs, outputs, useDueling=False):
        super(DQN, self).__init__()

        self.input_size = inputs
        self.output_size = outputs
        self.useDueling = useDueling

        # Define the common features layers (fully connected layers)
        self.features = nn.Sequential(
            nn.Linear(self.input_size, 128),
            nn.ReLU(),
            nn.Linear(128, 128),
            nn.ReLU()
        )

        # Define the layers for standard DQN
        self.layers = nn.Sequential(
            nn.Linear(128, 128),
            nn.ReLU(),
            nn.Linear(128, self.output_size)
        )

        # Define the layers for Dueling DQN
        self.layersdueling = nn.Sequential(
            nn.Linear(128, 128),
            nn.ReLU()
        )
        self.advantage = nn.Sequential(
            nn.Linear(128, self.output_size)
        )
        self.value = nn.Sequential(
            nn.Linear(128, 1)
        )

    def forward(self, x):
        x = x.view(x.size(0), -1)
        x = self.features(x)
        x = x.view(x.size(0), -1)

        if self.useDueling:
            x = self.layersdueling(x)
            advantage = self.advantage(x)
            value = self.value(x)
            return value + advantage - advantage.mean()
        else:
            return self.layers(x)

    def policy(self, state):
        with torch.no_grad():
            return self.__call__(state.unsqueeze(0)).argmax()

    def getPolicy(self, state, eps_threshold):
        sample = random.random()
        if sample > eps_threshold:
            with torch.no_grad():
                return self.__call__(state.unsqueeze(0)).argmax()
        else:
            return torch.tensor(random.randrange(self.output_size), device=device, dtype=torch.long)


# Simple Robot Environment




###    Description:
        Discrete control task to learn from affordances or image-like data, a top-down racing environment.
###    Observation:
        Types: PoseOnly, PoseAndLocalMap, GlobalMap, PoseAndLocalMapInline
        **PoseOnly** -> 3 values (x,y direction to the goal and robot's yaw)
        **PoseAndLocalMapInline** -> 3+NxN values (x,y direction to the goal and robot's yaw and NxN local map inline)
        (The following modes may not work)
        **PoseAndLocalMap** -> 3 values (x,y direction to the goal and robot's yaw) and local map ("image") with size (N,N)
        **GlobalMap** -> 3 values (robot's x,y and yaw) and global map ("images") with size (M,M)

        

###    Actions:
       Check the **action_mapping** variable on **def step(self, action):**.
       By default, 3 actions are available.
        

###    Reward:
        The reward is the negative distance to the goal, with an increasing penalization as the number of steps increases. Collisions provide a small reward, and when reaching the goal the robot receives a high  reward.

###    Starting State:
        The robot starts at the pose defined by **def setPosition(self,position):** and tries to reach the goal position defined set by **def setGoal(self,goal):**.

###    Episode Termination:
        
1. Robot reaches the goal.
2. Robot collides
3. The goal is not reached after exploring the maximum number of steps.

###     Solved Requirements:
        Consistently reach the goal position.


# DQN movie generation (for visual evaluation in Google Colab)

In [6]:
from gymnasium.wrappers.record_video import RecordVideo
def createMovie(Network,Filename):
    env.render_mode="rgb_array"
    new_state, _ = env.reset(options={})#,seed=6)  # weird behaviour using the gym API

    env1 = env
    envX = RecordVideo(env1, video_folder='./videos/'+Filename,  episode_trigger = lambda episode_number: True)

    new_state, _ = envX.reset(options={})#,seed=6)
    envX.start_video_recorder()

    if(envX.unwrapped.mode==Modes.PoseAndLocalMapInline):
      stackedStateX=[new_state for i in range(FrameStack)];
      state=torch.from_numpy(np.reshape(np.array(stackedStateX),(FrameStack,3+envX.unwrapped.local_grid_size*envX.unwrapped.local_grid_size,1))).float().to(device)


    elif(envX.unwrapped.mode==Modes.PoseOnly):
      stackedStateX=[new_state for i in range(FrameStack)];
      state=torch.from_numpy(np.reshape(np.array(stackedStateX),(FrameStack,3,1))).float().to(device)

    elif(envX.unwrapped.mode==Modes.PoseAndLocalMap):
      stackedStateX=[np.rollaxis(new_state[1], 2, 0).copy() for i in range(FrameStack)];
      state=torch.from_numpy(np.reshape(np.array(stackedStateX),(FrameStack,envX.unwrapped.local_grid_size,envX.unwrapped.local_grid_size))).float().to(device)
      stackedStatepose=[new_state[0] for i in range(FrameStack)];
      statepose=torch.from_numpy(np.reshape(np.array(stackedStatepose),(FrameStack,3,1))).float().to(device)

    elif(envX.unwrapped.mode==Modes.GlobalMap):
      stackedStateX=[np.rollaxis(new_state, 2, 0).copy() for i in range(FrameStack)];
      state=torch.from_numpy(np.reshape(np.array(stackedStateX),(FrameStack,envX.unwrapped.grid_size,envX.unwrapped.grid_size))).float().to(device)


    Network.eval()
    resetStatus=0;


    while True:
        envX.render()

        action = Network.policy(state);
        reward=0
        for x in range(ControlSteps):
            new_state, r, done, truncated, _  = envX.step(action.item())
            reward+=r
            for i in range(FPS-1):
                envX.step(-1)
            if(done or truncated):
                break



        stackedStateX.pop(0)



        if(envX.unwrapped.mode==Modes.PoseAndLocalMapInline):
          stackedStateX.append(new_state.copy())
          state=torch.from_numpy(np.reshape(np.array(stackedStateX),(FrameStack,3+envX.unwrapped.local_grid_size*envX.unwrapped.local_grid_size,1))).float().to(device)

        elif(envX.unwrapped.mode==Modes.PoseOnly):
          stackedStateX.append(new_state.copy())
          state=torch.from_numpy(np.reshape(np.array(stackedStateX),(FrameStack,3,1))).float().to(device)

        elif(envX.unwrapped.mode==Modes.PoseAndLocalMap):
          stackedStateX.append(np.rollaxis(new_state[1], 2, 0).copy())
          state=torch.from_numpy(np.reshape(np.array(stackedStateX),(FrameStack,envX.unwrapped.local_grid_size,envX.unwrapped.local_grid_size))).float().to(device)
          stackedStatepose.pop(0)
          stackedStatepose.append(new_state[0].copy())
          statepose=torch.from_numpy(np.reshape(np.array(stackedStatepose),(FrameStack,3,1))).float().to(device)

        elif(envX.unwrapped.mode==Modes.GlobalMap):
          stackedStateX.append(np.rollaxis(new_state, 2, 0).copy())
          state=torch.from_numpy(np.reshape(np.array(stackedStateX),(FrameStack,envX.unwrapped.grid_size,envX.unwrapped.grid_size))).float().to(device)

        if done or truncated: #or resetStatus>=ResetCounter:
          break;

    envX.close()
    env1.close();
    Network.train()
    mp4list = glob.glob('./videos/'+Filename+'/*.mp4')
    if len(mp4list) > 0:
        mp4 = mp4list[0]
        video = io.open(mp4, 'r+b').read()
        encoded = base64.b64encode(video)
        ipythondisplay.display(HTML(data='''<video alt="test" autoplay
                loop controls style="height: 400px;">
                <source src="data:video/mp4;base64,{0}" type="video/mp4" />
             </video>'''.format(encoded.decode('ascii'))))
    else:
        print("Could not find video")

# Dummy Net with random policy

In [7]:
class RandomNet(nn.Module):

    def __init__(self,outputs):
        super(RandomNet, self).__init__()
        self.output_size=outputs;


    def forward(self, x):
        return  x

    def policy(self,state):
       return  torch.tensor([[random.randrange(self.output_size)]], device=device, dtype=torch.long)


# Parameters

In [8]:

n_actions=3

FPS=30
#hyper-parameters
TotalEpisodes=500;
MaxSteps=150;
ControlSteps=1; # number of iterations the same action is executed in the environment
FrameStack=1 # number of consecutive frames used to represent the state
FreezeCounter=25; # clone the model every X episodes
BatchSize=256;
exploration_threshold=1
exploration_threshold_min=0.01
exploration_decay=0.0025
discount_factor=0.95
LearningRate=0.001

BufferSize=200000  # max number of experiences in the buffer (may need to be smaller if CUDA memory errors occur).

# use these flags to enable Double DQN and Duelinng (Dueling DQN or Double Dueling DQN)
usedoubleDQN=False
useDueling=False

env.max_steps=MaxSteps


Run the new network with random policy

In [9]:
randomnet=RandomNet(n_actions)
#createMovie(randomnet,"random")

# Initialization and Parameters:


In [10]:
#network DQN
policy_net = DQN(3*FrameStack, n_actions).to(device)
target_net = DQN(3*FrameStack, n_actions).to(device)
target_net.load_state_dict(policy_net.state_dict())
target_net.eval()

buffer = ReplayBuffer(BufferSize);
optimizer = torch.optim.Adam(policy_net.parameters(), lr=LearningRate)
loss=  torch.nn.SmoothL1Loss()


# DQN training

In [11]:
def trainModel(buffer_data):
    if buffer_data.counter < BatchSize:
        return 0.0

    state_batch, action_batch, reward_batch, new_state_batch, done_batch = buffer_data.sample_buffer(BatchSize)
    state_batch = torch.stack(state_batch).to(device)
    action_batch = torch.stack(action_batch).reshape(-1,1).to(device)
    new_state_batch = torch.stack(new_state_batch).to(device)
    reward_batch = torch.stack(reward_batch).reshape(-1,1).to(device)
    done_batch = torch.stack(done_batch).reshape(-1,1).to(device)


    if usedoubleDQN:
      next_state_actions = policy_net(new_state_batch).argmax(dim=1, keepdim=True)
      q_max_next = target_net(new_state_batch).gather(1, next_state_actions)
    else:
        q_actual = torch.gather(policy_net(state_batch),1,index=action_batch)
        with torch.no_grad():
            q_max_next, _ = target_net(new_state_batch).max(dim=1)
            q_max_next=q_max_next.reshape(-1, 1)

        q_target = (q_max_next * discount_factor)*(1-done_batch) + reward_batch
        ll=loss(q_actual, q_target)

    optimizer.zero_grad()
    ll.backward()
    torch.nn.utils.clip_grad_norm_(policy_net.parameters(), 1)
    optimizer.step()

    return ll.item();

In [12]:
import torch

def trainModel(buffer_data, policy_net, target_net, optimizer, loss, BatchSize, discount_factor, usedoubleDQN):
    if buffer_data.counter < BatchSize:
        return 0.0

    # Sample a batch from the replay buffer
    state_batch, action_batch, reward_batch, new_state_batch, done_batch = buffer_data.sample_buffer(BatchSize)
    state_batch = torch.stack(state_batch).to(device)
    action_batch = torch.stack(action_batch).reshape(-1, 1).to(device)
    new_state_batch = torch.stack(new_state_batch).to(device)
    reward_batch = torch.stack(reward_batch).reshape(-1, 1).to(device)
    done_batch = torch.stack(done_batch).reshape(-1, 1).to(device)

    # Calculate the current Q values
    q_actual = torch.gather(policy_net(state_batch), 1, index=action_batch)

    with torch.no_grad():
        if usedoubleDQN:
            # For Double DQN
            next_state_actions = policy_net(new_state_batch).argmax(dim=1, keepdim=True)
            q_max_next = target_net(new_state_batch).gather(1, next_state_actions)
        else:
            # For standard DQN
            q_max_next, _ = target_net(new_state_batch).max(dim=1, keepdim=True)

        # Calculate the target Q values
        q_target = (q_max_next * discount_factor) * (1 - done_batch) + reward_batch

    # Compute the loss
    ll = loss(q_actual, q_target)

    # Optimize the model
    optimizer.zero_grad()
    ll.backward()
    torch.nn.utils.clip_grad_norm_(policy_net.parameters(), 1)
    optimizer.step()

    return ll.item()


# First Scenario

In [30]:
# Usage example:
env = APA2023SimpleRobotEnv(grid_size=50,local_grid_size=11,mode=Modes.PoseAndLocalMapInline)
observation = env.reset()
env.setGoal(np.array([5, 5]))
env.setPosition(np.array([-2,-2.5,0 ]))

env.max_steps = 10

env.setPosition(np.array([-2,-2.5,0 ]))
createMovie(randomnet,'randomnet')

Moviepy - Building video /content/videos/randomnet/rl-video-episode-0.mp4.
Moviepy - Writing video /content/videos/randomnet/rl-video-episode-0.mp4



Moviepy - Done !
Moviepy - video ready /content/videos/randomnet/rl-video-episode-0.mp4


Moviepy - Building video /content/videos/randomnet/rl-video-episode-0.mp4.
Moviepy - Writing video /content/videos/randomnet/rl-video-episode-0.mp4



Moviepy - Done !
Moviepy - video ready /content/videos/randomnet/rl-video-episode-0.mp4


In [13]:
import copy


loss_val,scores, episodes,events, avg_scores,avg_scores20,exploration = [],[],[], [],[], [], []

bestScore=-99999;
at=0;
bestNet=copy.deepcopy(policy_net);


for f in range(TotalEpisodes):
    done  = False
    score = 0.0
    tloss = 0.0

    new_state,_=env.reset()

    if(env.mode==Modes.PoseAndLocalMapInline):
      stackedStateX=[new_state for i in range(FrameStack)];
      state=torch.from_numpy(np.reshape(np.array(stackedStateX),(FrameStack,3+env.local_grid_size*env.local_grid_size,1))).float().to(device)

    elif(env.mode==Modes.PoseOnly):
      stackedStateX=[new_state for i in range(FrameStack)];
      state=torch.from_numpy(np.reshape(np.array(stackedStateX),(FrameStack,3,1))).float().to(device)
      print("111 ",len(stackedStateX),FrameStack)

    elif(env.mode==Modes.PoseAndLocalMap):
      stackedStateX=[np.rollaxis(new_state, 2, 0).copy() for i in range(FrameStack)];
      state=torch.from_numpy(np.reshape(np.array(stackedStateX),(FrameStack,env.local_grid_size,env.local_grid_size))).float().to(device)
      stackedStatepose=[new_state[0] for i in range(FrameStack)];
      statepose=torch.from_numpy(np.reshape(np.array(stackedStatepose),(FrameStack,3,1))).float().to(device)

    elif(env.mode==Modes.GlobalMap):
      stackedStateX=[np.rollaxis(new_state, 2, 0).copy() for i in range(FrameStack)];
      state=torch.from_numpy(np.reshape(np.array(stackedStateX),(FrameStack,env.grid_size,env.grid_size))).float().to(device)



    if f % FreezeCounter == 0:
       print("########################################"+str(f)+" of "+str(TotalEpisodes), exploration_threshold)
       target_net.load_state_dict(policy_net.state_dict())

    resetStatus=0;
    i=0

    for F in range(MaxSteps):
        action = policy_net.getPolicy(state,exploration_threshold)
        reward=0
        for _ in range(ControlSteps):
            new_state, r, done,trunc, info = env.step(action.item())
            reward+=r
            i=i+1
            if(done or trunc):
                break

        oldstate=state.clone();
        stackedStateX.pop(0)


        if(env.mode==Modes.PoseAndLocalMapInline):
          stackedStateX.append(new_state.copy())
          state=torch.from_numpy(np.reshape(np.array(stackedStateX),(FrameStack,3+env.local_grid_size*env.local_grid_size,1))).float().to(device)

        elif(env.mode==Modes.PoseOnly):
          stackedStateX.append(new_state.copy())
          state=torch.from_numpy(np.reshape(np.array(stackedStateX),(FrameStack,3,1))).float().to(device)

        elif(env.mode==Modes.PoseAndLocalMap):
          stackedStateX.append(np.rollaxis(new_state[1], 2, 0).copy())
          state=torch.from_numpy(np.reshape(np.array(stackedStateX),(FrameStack,env.local_grid_size,env.local_grid_size))).float().to(device)
          stackedStatepose.pop(0)
          stackedStatepose.append(new_state[0].copy())
          statepose=torch.from_numpy(np.reshape(np.array(stackedStatepose),(FrameStack,3,1))).float().to(device)

        elif(env.mode==Modes.GlobalMap):
          stackedStateX.append(np.rollaxis(new_state, 2, 0).copy())
          state=torch.from_numpy(np.reshape(np.array(stackedStateX),(FrameStack,env.grid_size,env.grid_size))).float().to(device)


        score += reward
        if(F<(MaxSteps-1)):
            buffer.store_tuples(oldstate, action, torch.tensor(reward), state, torch.tensor(int(done)))

        trainModel(buffer,policy_net,target_net,optimizer,loss,BatchSize,discount_factor,False)
        if(done or trunc):
            break
    exploration_threshold= exploration_threshold-exploration_decay if exploration_threshold > exploration_threshold_min else exploration_threshold_min
    print("EP: ",score," It: ",F," eth: ",exploration_threshold)

    if(score>bestScore and exploration_threshold<0.1):
        print("Best EP: ",score," It: ",F," eth: ",exploration_threshold)
        bestScore=score;
        bestNet=copy.deepcopy(policy_net);
        at=f;

    exploration.append(exploration_threshold)
    scores.append(score)
    episodes.append(f)
    events.append(F)
    avg_scores.append(score/F)
    avg_scores20.append(np.mean(scores[-20:]))


torch.save(bestNet.state_dict(), "BestRobot_model.ckpt")

111  1 1
########################################0 of 500 1
EP:  -263.77421625360967  It:  149  eth:  0.9975
111  1 1
EP:  -1140.501060162061  It:  88  eth:  0.9950000000000001
111  1 1
EP:  -225.71667962261373  It:  149  eth:  0.9925000000000002
111  1 1
EP:  -241.3797579519618  It:  149  eth:  0.9900000000000002
111  1 1
EP:  -1102.421271360275  It:  74  eth:  0.9875000000000003
111  1 1
EP:  -214.04177478452422  It:  149  eth:  0.9850000000000003
111  1 1
EP:  -1162.1768315852696  It:  103  eth:  0.9825000000000004
111  1 1
EP:  -1182.0825687358583  It:  114  eth:  0.9800000000000004
111  1 1
EP:  -1181.0370482455046  It:  111  eth:  0.9775000000000005
111  1 1
EP:  -1163.4718371476024  It:  99  eth:  0.9750000000000005
111  1 1
EP:  -232.20691122137708  It:  149  eth:  0.9725000000000006
111  1 1
EP:  -1194.3705833115453  It:  137  eth:  0.9700000000000006
111  1 1
EP:  -220.2512733057667  It:  149  eth:  0.9675000000000007
111  1 1
EP:  -1149.6290468441591  It:  117  eth:  0.96500

KeyboardInterrupt: ignored

# Plot performance

In [ ]:
import matplotlib
matplotlib.use('module://matplotlib_inline.backend_inline')
plt.figure(figsize=(12, 6), dpi=80)
plt.plot(episodes, scores)
plt.plot(episodes, events)
plt.plot(episodes, avg_scores)
plt.plot(episodes, avg_scores20)
plt.plot(episodes, exploration)
plt.xlabel('episodes')
plt.ylabel('y axis label')

plt.title('Track Locked DQN '+str(usedoubleDQN)+' '+str(useDueling))
plt.legend(['scores', 'events', 'avg_scores', 'avg_scores100','exploration'])

plt.show()

In [ ]:
#env.setGoal(np.array([5, 5]))
#env.setPosition(np.array([-2,-2.5,0 ]))
createMovie(bestNet,'bestNet')




# Second Scenario

In [14]:
# Usage example:
env = APA2023SimpleRobotEnv(grid_size=50,local_grid_size=11,mode=Modes.PoseAndLocalMapInline)
observation = env.reset()
env.setGoal(np.array([5, 5]))
env.setPosition(np.array([-2,-2.5,0 ]))


env.addObstacleLine(np.array([0,-2]),np.array([ -5.9,-2]),1)
env.addObstacleLine(np.array([0, -2]),np.array([0, 5.9]),1)
env.addObstacleLine(np.array([2,-4]),np.array([ -5.9,-4]),1)
env.addObstacleLine(np.array([2, -4]),np.array([2, 4]),1)
env.addObstacleLine(np.array([2,4]),np.array([ 5.9,4]),1)

env.max_steps = 10

env.setPosition(np.array([-2,-2.5,0 ]))
createMovie(randomnet,'randomnet')


Moviepy - Building video /content/videos/randomnet/rl-video-episode-0.mp4.
Moviepy - Writing video /content/videos/randomnet/rl-video-episode-0.mp4



Moviepy - Done !
Moviepy - video ready /content/videos/randomnet/rl-video-episode-0.mp4
Moviepy - Building video /content/videos/randomnet/rl-video-episode-0.mp4.
Moviepy - Writing video /content/videos/randomnet/rl-video-episode-0.mp4



Moviepy - Done !
Moviepy - video ready /content/videos/randomnet/rl-video-episode-0.mp4


In [ ]:
import copy


#hyper-parameters
TotalEpisodes=15000;
MaxSteps=300;
ControlSteps=1; # number of iterations the same action is executed in the environment
FrameStack=1 # number of consecutive frames used to represent the state
FreezeCounter=25; # clone the model every X episodes
BatchSize=512;
exploration_threshold=1
exploration_threshold_min=0.01
exploration_decay=0.0013/5
discount_factor=0.99
LearningRate=0.0001
BufferSize=200000

env.max_steps=MaxSteps

#network DQN
policy_net = DQN((3+11*11)*FrameStack, n_actions).to(device)
target_net = DQN((3+11*11)*FrameStack, n_actions).to(device)
target_net.load_state_dict(policy_net.state_dict())
target_net.eval()

buffer = ReplayBuffer(BufferSize);
optimizer = torch.optim.Adam(policy_net.parameters(), lr=LearningRate)
loss=  torch.nn.SmoothL1Loss()

loss_val,scores, episodes,events, avg_scores,avg_scores20,exploration = [],[],[], [],[], [], []

bestScore=-99999;
at=0;
bestNet=copy.deepcopy(policy_net);


for f in range(TotalEpisodes):
    done  = False
    score = 0.0
    tloss = 0.0

    new_state,_=env.reset()

    if(env.mode==Modes.PoseAndLocalMapInline):
      stackedStateX=[new_state for i in range(FrameStack)];
      state=torch.from_numpy(np.reshape(np.array(stackedStateX),(FrameStack,3+env.local_grid_size*env.local_grid_size,1))).float().to(device)

    if(env.mode==Modes.PoseOnly ):
      stackedStateX=[new_state for i in range(FrameStack)];
      state=torch.from_numpy(np.reshape(np.array(stackedStateX),(FrameStack,3,1))).float().to(device)

    if(env.mode==Modes.PoseAndLocalMap):
      stackedStateX=[np.rollaxis(new_state[1], 2, 0).copy() for i in range(FrameStack)];
      state=torch.from_numpy(np.reshape(np.array(stackedStateX),(FrameStack,env.local_grid_size,env.local_grid_size))).float().to(device)
      stackedStatepose=[new_state[0] for i in range(FrameStack)];
      statepose=torch.from_numpy(np.reshape(np.array(stackedStatepose),(FrameStack,3,1))).float().to(device)

    if(env.mode==Modes.GlobalMap):
      stackedStateX=[np.rollaxis(new_state, 2, 0).copy() for i in range(FrameStack)];
      state=torch.from_numpy(np.reshape(np.array(stackedStateX),(FrameStack,env.grid_size,env.grid_size))).float().to(device)


    if f % FreezeCounter == 0:
       print("########################################"+str(f)+" of "+str(TotalEpisodes), exploration_threshold)
       target_net.load_state_dict(policy_net.state_dict())

    resetStatus=0;
    i=0

    for F in range(MaxSteps):
        action = policy_net.getPolicy(state,exploration_threshold)
        reward=0
        for _ in range(ControlSteps):
            new_state, r, done,trunc, info = env.step(action.item())
            reward+=r
            i=i+1
            if(done or trunc):
                break

        oldstate=state.clone();
        stackedStateX.pop(0)

        if(env.mode==Modes.PoseAndLocalMapInline):
          stackedStateX.append(new_state.copy())
          state=torch.from_numpy(np.reshape(np.array(stackedStateX),(FrameStack,3+env.local_grid_size*env.local_grid_size,1))).float().to(device)

        if(env.mode==Modes.PoseOnly):
          stackedStateX.append(new_state.copy())
          state=torch.from_numpy(np.reshape(np.array(stackedStateX),(FrameStack,3,1))).float().to(device)

        if(env.mode==Modes.PoseAndLocalMap):
          stackedStateX.append(np.rollaxis(new_state[1], 2, 0).copy())
          state=torch.from_numpy(np.reshape(np.array(stackedStateX),(FrameStack,env.local_grid_size,env.local_grid_size))).float().to(device)
          stackedStatepose.pop(0)
          stackedStatepose.append(new_state[0].copy())
          statepose=torch.from_numpy(np.reshape(np.array(stackedStatepose),(FrameStack,3,1))).float().to(device)

        if(env.mode==Modes.GlobalMap):
          stackedStateX.append(np.rollaxis(new_state, 2, 0).copy())
          state=torch.from_numpy(np.reshape(np.array(stackedStateX),(FrameStack,env.grid_size,env.grid_size))).float().to(device)


        score += reward
        if(F<(MaxSteps-1)):
            buffer.store_tuples(oldstate, action, torch.tensor(reward), state, torch.tensor(int(done)))

        trainModel(buffer,policy_net,target_net,optimizer,loss,BatchSize,discount_factor,False)


        if(done or trunc):
            break
    exploration_threshold= exploration_threshold-exploration_decay if exploration_threshold > exploration_threshold_min else exploration_threshold_min
    print("EP: ",score," It: ",F," eth: ",exploration_threshold)

    if(score>bestScore and exploration_threshold<0.1):
        print("Best EP: ",score," It: ",F," eth: ",exploration_threshold)
        bestScore=score;
        torch.save(policy_net.state_dict(), 'BestRobot_model.ckpt')
        at=f;

    exploration.append(exploration_threshold)
    scores.append(score)
    episodes.append(f)
    events.append(F)
    avg_scores.append(score/F)
    avg_scores20.append(np.mean(scores[-20:]))




# *Plot* performance

In [ ]:
import matplotlib
matplotlib.use('module://matplotlib_inline.backend_inline')
plt.figure(figsize=(12, 6), dpi=80)
plt.plot(episodes, scores)
plt.plot(episodes, events)
plt.plot(episodes, avg_scores)
plt.plot(episodes, avg_scores20)
plt.plot(episodes, exploration)
plt.xlabel('episodes')
plt.ylabel('y axis label')

plt.title('Track Locked DQN '+str(usedoubleDQN)+' '+str(useDueling))
plt.legend(['scores', 'events', 'avg_scores', 'avg_scores100','exploration'])

plt.show()

In [ ]:
policy_net.load_state_dict(torch.load('BestRobot_model.ckpt'))
createMovie(policy_net,'bestNet')


# Third Scenario


In [44]:
# Create a new environment instance
env = APA2023SimpleRobotEnv(grid_size=50, local_grid_size=11, mode=Modes.PoseAndLocalMapInline)
env.reset()

# Set a new goal and starting position
env.setPosition(np.array([4, -4, 0]))

env.addObstacleLine(np.array([-5, -5]), np.array([5, -5]), 1)  # Bottom border
env.addObstacleLine(np.array([-5, 5]), np.array([5, 5]), 1)    # Top border
env.addObstacleLine(np.array([-5, -5]), np.array([-5, 5]), 1)  # Left border
env.addObstacleLine(np.array([5, -5]), np.array([5, 5]), 1)    # Right border

# Internal walls to form a simple maze
env.addObstacleLine(np.array([-3, -5]), np.array([-3, 0]), 1)  # Vertical line on left side
env.addObstacleLine(np.array([3, 5]), np.array([3, 0]), 1)     # Vertical line on right side
env.addObstacleLine(np.array([-5, 2]), np.array([3, 2]), 1)    # Horizontal line on bottom side
env.addObstacleLine(np.array([5, -2]), np.array([0, -2]), 1)   # Horizontal line on top side

# Set a starting position and a goal
env.setGoal(np.array([4, 4]))
# Adjust other parameters if needed
env.max_steps = 10

# You can proceed with simulation or creating movie as needed
# createMovie(new_net, 'new_simulation')

createMovie(randomnet,'randomnet')

Moviepy - Building video /content/videos/randomnet/rl-video-episode-0.mp4.
Moviepy - Writing video /content/videos/randomnet/rl-video-episode-0.mp4



Moviepy - Done !
Moviepy - video ready /content/videos/randomnet/rl-video-episode-0.mp4


Moviepy - Building video /content/videos/randomnet/rl-video-episode-0.mp4.
Moviepy - Writing video /content/videos/randomnet/rl-video-episode-0.mp4



Moviepy - Done !
Moviepy - video ready /content/videos/randomnet/rl-video-episode-0.mp4


In [24]:
import copy


#hyper-parameters
TotalEpisodes= 30000;
MaxSteps=600;
ControlSteps=1; # number of iterations the same action is executed in the environment
FrameStack=1 # number of consecutive frames used to represent the state
FreezeCounter = 200; # clone the model every X episodes
BatchSize=512;
exploration_threshold=1
exploration_threshold_min=0.001
exploration_decay=0.0013/20
discount_factor=0.999
LearningRate=0.0001
env.max_steps=MaxSteps

#network DQN
policy_net = DQN((3+11*11)*FrameStack, n_actions).to(device)
target_net = DQN((3+11*11)*FrameStack, n_actions).to(device)
target_net.load_state_dict(policy_net.state_dict())
target_net.eval()

buffer = ReplayBuffer(BufferSize);
optimizer = torch.optim.Adam(policy_net.parameters(), lr=LearningRate)
loss=  torch.nn.SmoothL1Loss()

loss_val,scores, episodes,events, avg_scores,avg_scores20,exploration = [],[],[], [],[], [], []

bestScore=-99999;
at=0;
bestNet=copy.deepcopy(policy_net);


for f in range(TotalEpisodes):
    done  = False
    score = 0.0
    tloss = 0.0

    new_state,_=env.reset()

    if(env.mode==Modes.PoseAndLocalMapInline):
      stackedStateX=[new_state for i in range(FrameStack)];
      state=torch.from_numpy(np.reshape(np.array(stackedStateX),(FrameStack,3+env.local_grid_size*env.local_grid_size,1))).float().to(device)

    if(env.mode==Modes.PoseOnly ):
      stackedStateX=[new_state for i in range(FrameStack)];
      state=torch.from_numpy(np.reshape(np.array(stackedStateX),(FrameStack,3,1))).float().to(device)

    if(env.mode==Modes.PoseAndLocalMap):
      stackedStateX=[np.rollaxis(new_state[1], 2, 0).copy() for i in range(FrameStack)];
      state=torch.from_numpy(np.reshape(np.array(stackedStateX),(FrameStack,env.local_grid_size,env.local_grid_size))).float().to(device)
      stackedStatepose=[new_state[0] for i in range(FrameStack)];
      statepose=torch.from_numpy(np.reshape(np.array(stackedStatepose),(FrameStack,3,1))).float().to(device)

    if(env.mode==Modes.GlobalMap):
      stackedStateX=[np.rollaxis(new_state, 2, 0).copy() for i in range(FrameStack)];
      state=torch.from_numpy(np.reshape(np.array(stackedStateX),(FrameStack,env.grid_size,env.grid_size))).float().to(device)


    if f % FreezeCounter == 0:
       print("########################################"+str(f)+" of "+str(TotalEpisodes), exploration_threshold)
       target_net.load_state_dict(policy_net.state_dict())

    resetStatus=0;
    i=0

    for F in range(MaxSteps):
        action = policy_net.getPolicy(state,exploration_threshold)
        reward=0
        for _ in range(ControlSteps):
            new_state, r, done,trunc, info = env.step(action.item())
            reward+=r
            i=i+1
            if(done or trunc):
                break

        oldstate=state.clone();
        stackedStateX.pop(0)

        if(env.mode==Modes.PoseAndLocalMapInline):
          stackedStateX.append(new_state.copy())
          state=torch.from_numpy(np.reshape(np.array(stackedStateX),(FrameStack,3+env.local_grid_size*env.local_grid_size,1))).float().to(device)

        if(env.mode==Modes.PoseOnly):
          stackedStateX.append(new_state.copy())
          state=torch.from_numpy(np.reshape(np.array(stackedStateX),(FrameStack,3,1))).float().to(device)

        if(env.mode==Modes.PoseAndLocalMap):
          stackedStateX.append(np.rollaxis(new_state[1], 2, 0).copy())
          state=torch.from_numpy(np.reshape(np.array(stackedStateX),(FrameStack,env.local_grid_size,env.local_grid_size))).float().to(device)
          stackedStatepose.pop(0)
          stackedStatepose.append(new_state[0].copy())
          statepose=torch.from_numpy(np.reshape(np.array(stackedStatepose),(FrameStack,3,1))).float().to(device)

        if(env.mode==Modes.GlobalMap):
          stackedStateX.append(np.rollaxis(new_state, 2, 0).copy())
          state=torch.from_numpy(np.reshape(np.array(stackedStateX),(FrameStack,env.grid_size,env.grid_size))).float().to(device)


        score += reward
        if(F<(MaxSteps-1)):
            buffer.store_tuples(oldstate, action, torch.tensor(reward), state, torch.tensor(int(done)))

        trainModel(buffer,policy_net,target_net,optimizer,loss,BatchSize,discount_factor,False)


        if(done or trunc):
            break
    exploration_threshold= exploration_threshold-exploration_decay if exploration_threshold > exploration_threshold_min else exploration_threshold_min
    print("EP: ",score," It: ",F," eth: ",exploration_threshold)

    if(score>bestScore and exploration_threshold<0.1):
        print("Best EP: ",score," It: ",F," eth: ",exploration_threshold)
        bestScore=score;
        torch.save(policy_net.state_dict(), 'BestRobot_model.ckpt')
        at=f;

    exploration.append(exploration_threshold)
    scores.append(score)
    episodes.append(f)
    events.append(F)
    avg_scores.append(score/F)
    avg_scores20.append(np.mean(scores[-20:]))




########################################0 of 20000 1
EP:  -1012.4786516030765  It:  10  eth:  0.99987
EP:  -1011.7976289877487  It:  9  eth:  0.9997400000000001
EP:  -1012.4801292860423  It:  10  eth:  0.9996100000000001
EP:  -1049.3554880157544  It:  39  eth:  0.9994800000000001
EP:  -1012.5023448586035  It:  10  eth:  0.9993500000000002
EP:  -1015.31987681028  It:  12  eth:  0.9992200000000002
EP:  -1014.7924439587962  It:  12  eth:  0.9990900000000003
EP:  -1012.5381914752049  It:  10  eth:  0.9989600000000003
EP:  -1013.7105379169609  It:  11  eth:  0.9988300000000003
EP:  -1016.2707500797641  It:  13  eth:  0.9987000000000004
EP:  -1016.298840461502  It:  13  eth:  0.9985700000000004
EP:  -1019.7463943923407  It:  15  eth:  0.9984400000000004
EP:  -1012.5722966986767  It:  10  eth:  0.9983100000000005
EP:  -1043.3646145961407  It:  33  eth:  0.9981800000000005
EP:  -1018.1608904411444  It:  15  eth:  0.9980500000000005
EP:  -1013.7099140340727  It:  11  eth:  0.9979200000000006
EP

KeyboardInterrupt: ignored

In [ ]:
import matplotlib
matplotlib.use('module://matplotlib_inline.backend_inline')
plt.figure(figsize=(12, 6), dpi=80)
plt.plot(episodes, scores)
plt.plot(episodes, events)
plt.plot(episodes, avg_scores)
plt.plot(episodes, avg_scores20)
plt.plot(episodes, exploration)
plt.xlabel('episodes')
plt.ylabel('y axis label')

plt.title('Track Locked DQN '+str(usedoubleDQN)+' '+str(useDueling))
plt.legend(['scores', 'events', 'avg_scores', 'avg_scores100','exploration'])

plt.show()

In [ ]:
policy_net.load_state_dict(torch.load('BestRobot_model.ckpt'))
createMovie(policy_net,'bestNet')


# Fourth Scenario


In [42]:
# Create a new environment instance
env = APA2023SimpleRobotEnv(grid_size=50, local_grid_size=11, mode=Modes.PoseAndLocalMapInline)
env.reset()

env.setPosition(np.array([-4, -4, 0]))

# Borders
env.addObstacleLine(np.array([-5, -5]), np.array([5, -5]), 1)  # Bottom
env.addObstacleLine(np.array([-5, 5]), np.array([5, 5]), 1)    # Top
env.addObstacleLine(np.array([-5, -5]), np.array([-5, 5]), 1)  # Left
env.addObstacleLine(np.array([5, -5]), np.array([5, 5]), 1)    # Right

# Maze Obstacles
env.addObstacleLine(np.array([-1, -5]), np.array([-1, -1]), 1) # Left Lower Vertical
env.addObstacleLine(np.array([1, 5]), np.array([1, 1]), 1)     # Right Upper Vertical
env.addObstacleLine(np.array([-5, 1]), np.array([-1, 1]), 1)   # Bottom Horizontal
env.addObstacleLine(np.array([5, -1]), np.array([3, -1]), 1)   # Top Horizontal

# Goal and Start Position
env.setGoal(np.array([4, 4]))
# Adjust other parameters if needed
env.max_steps = 10

# You can proceed with simulation or creating movie as needed
# createMovie(new_net, 'new_simulation')

createMovie(randomnet,'randomnet')

Moviepy - Building video /content/videos/randomnet/rl-video-episode-0.mp4.
Moviepy - Writing video /content/videos/randomnet/rl-video-episode-0.mp4



Moviepy - Done !
Moviepy - video ready /content/videos/randomnet/rl-video-episode-0.mp4


Moviepy - Building video /content/videos/randomnet/rl-video-episode-0.mp4.
Moviepy - Writing video /content/videos/randomnet/rl-video-episode-0.mp4



Moviepy - Done !
Moviepy - video ready /content/videos/randomnet/rl-video-episode-0.mp4


In [ ]:
import copy


#hyper-parameters
TotalEpisodes=20000;
MaxSteps=500;
ControlSteps=1; # number of iterations the same action is executed in the environment
FrameStack=1 # number of consecutive frames used to represent the state
FreezeCounter=25; # clone the model every X episodes
BatchSize=512;
exploration_threshold=1
exploration_threshold_min=0.01
exploration_decay=0.0013/10
discount_factor=0.999
LearningRate=0.0001
BufferSize = 20000
env.max_steps=MaxSteps

#network DQN
policy_net = DQN((3+11*11)*FrameStack, n_actions).to(device)
target_net = DQN((3+11*11)*FrameStack, n_actions).to(device)
target_net.load_state_dict(policy_net.state_dict())
target_net.eval()

buffer = ReplayBuffer(BufferSize);
optimizer = torch.optim.Adam(policy_net.parameters(), lr=LearningRate)
loss=  torch.nn.SmoothL1Loss()

loss_val,scores, episodes,events, avg_scores,avg_scores20,exploration = [],[],[], [],[], [], []

bestScore=-99999;
at=0;
bestNet=copy.deepcopy(policy_net);


for f in range(TotalEpisodes):
    done  = False
    score = 0.0
    tloss = 0.0

    new_state,_=env.reset()

    if(env.mode==Modes.PoseAndLocalMapInline):
      stackedStateX=[new_state for i in range(FrameStack)];
      state=torch.from_numpy(np.reshape(np.array(stackedStateX),(FrameStack,3+env.local_grid_size*env.local_grid_size,1))).float().to(device)

    if(env.mode==Modes.PoseOnly ):
      stackedStateX=[new_state for i in range(FrameStack)];
      state=torch.from_numpy(np.reshape(np.array(stackedStateX),(FrameStack,3,1))).float().to(device)

    if(env.mode==Modes.PoseAndLocalMap):
      stackedStateX=[np.rollaxis(new_state[1], 2, 0).copy() for i in range(FrameStack)];
      state=torch.from_numpy(np.reshape(np.array(stackedStateX),(FrameStack,env.local_grid_size,env.local_grid_size))).float().to(device)
      stackedStatepose=[new_state[0] for i in range(FrameStack)];
      statepose=torch.from_numpy(np.reshape(np.array(stackedStatepose),(FrameStack,3,1))).float().to(device)

    if(env.mode==Modes.GlobalMap):
      stackedStateX=[np.rollaxis(new_state, 2, 0).copy() for i in range(FrameStack)];
      state=torch.from_numpy(np.reshape(np.array(stackedStateX),(FrameStack,env.grid_size,env.grid_size))).float().to(device)


    if f % FreezeCounter == 0:
       print("########################################"+str(f)+" of "+str(TotalEpisodes), exploration_threshold)
       target_net.load_state_dict(policy_net.state_dict())

    resetStatus=0;
    i=0

    for F in range(MaxSteps):
        action = policy_net.getPolicy(state,exploration_threshold)
        reward=0
        for _ in range(ControlSteps):
            new_state, r, done,trunc, info = env.step(action.item())
            reward+=r
            i=i+1
            if(done or trunc):
                break

        oldstate=state.clone();
        stackedStateX.pop(0)

        if(env.mode==Modes.PoseAndLocalMapInline):
          stackedStateX.append(new_state.copy())
          state=torch.from_numpy(np.reshape(np.array(stackedStateX),(FrameStack,3+env.local_grid_size*env.local_grid_size,1))).float().to(device)

        if(env.mode==Modes.PoseOnly):
          stackedStateX.append(new_state.copy())
          state=torch.from_numpy(np.reshape(np.array(stackedStateX),(FrameStack,3,1))).float().to(device)

        if(env.mode==Modes.PoseAndLocalMap):
          stackedStateX.append(np.rollaxis(new_state[1], 2, 0).copy())
          state=torch.from_numpy(np.reshape(np.array(stackedStateX),(FrameStack,env.local_grid_size,env.local_grid_size))).float().to(device)
          stackedStatepose.pop(0)
          stackedStatepose.append(new_state[0].copy())
          statepose=torch.from_numpy(np.reshape(np.array(stackedStatepose),(FrameStack,3,1))).float().to(device)

        if(env.mode==Modes.GlobalMap):
          stackedStateX.append(np.rollaxis(new_state, 2, 0).copy())
          state=torch.from_numpy(np.reshape(np.array(stackedStateX),(FrameStack,env.grid_size,env.grid_size))).float().to(device)


        score += reward
        if(F<(MaxSteps-1)):
            buffer.store_tuples(oldstate, action, torch.tensor(reward), state, torch.tensor(int(done)))

        trainModel(buffer,policy_net,target_net,optimizer,loss,BatchSize,discount_factor,False)


        if(done or trunc):
            break
    exploration_threshold= exploration_threshold-exploration_decay if exploration_threshold > exploration_threshold_min else exploration_threshold_min
    print("EP: ",score," It: ",F," eth: ",exploration_threshold)

    if(score>bestScore and exploration_threshold<0.1):
        print("Best EP: ",score," It: ",F," eth: ",exploration_threshold)
        bestScore=score;
        torch.save(policy_net.state_dict(), 'BestRobot_model.ckpt')
        at=f;

    exploration.append(exploration_threshold)
    scores.append(score)
    episodes.append(f)
    events.append(F)
    avg_scores.append(score/F)
    avg_scores20.append(np.mean(scores[-20:]))




########################################0 of 30000 1
EP:  -1014.9364321633141  It:  14  eth:  0.999948
EP:  -1011.621102786572  It:  11  eth:  0.9998959999999999
EP:  -1011.6612428157957  It:  11  eth:  0.9998439999999998
EP:  -1010.5255434714248  It:  10  eth:  0.9997919999999998
EP:  -1012.451537230097  It:  12  eth:  0.9997399999999997
EP:  -1020.0075801868206  It:  19  eth:  0.9996879999999997
EP:  -1014.1155711321237  It:  14  eth:  0.9996359999999996
EP:  -1010.496754726197  It:  10  eth:  0.9995839999999996
EP:  -1009.3906509279018  It:  9  eth:  0.9995319999999995
EP:  -1014.9411376614217  It:  14  eth:  0.9994799999999995
EP:  -1010.460506773335  It:  10  eth:  0.9994279999999994
EP:  -1010.4661819656413  It:  10  eth:  0.9993759999999994
EP:  -1011.4902746633618  It:  11  eth:  0.9993239999999993
EP:  -1015.2251685874413  It:  15  eth:  0.9992719999999993
EP:  -1011.65964514351  It:  11  eth:  0.9992199999999992
EP:  -1011.6633784672412  It:  11  eth:  0.9991679999999992
EP: 

# plot Performance

In [ ]:
import matplotlib
matplotlib.use('module://matplotlib_inline.backend_inline')
plt.figure(figsize=(12, 6), dpi=80)
plt.plot(episodes, scores)
plt.plot(episodes, events)
plt.plot(episodes, avg_scores)
plt.plot(episodes, avg_scores20)
plt.plot(episodes, exploration)
plt.xlabel('episodes')
plt.ylabel('y axis label')

plt.title('Track Locked DQN '+str(usedoubleDQN)+' '+str(useDueling))
plt.legend(['scores', 'events', 'avg_scores', 'avg_scores100','exploration'])

plt.show()

In [ ]:
policy_net.load_state_dict(torch.load('BestRobot_model.ckpt'))
createMovie(policy_net,'bestNet')


# Fifth Scenario



In [48]:
# Create a new environment instance
env = APA2023SimpleRobotEnv(grid_size=50, local_grid_size=11, mode=Modes.PoseAndLocalMapInline)
env.reset()

# Define the borders of the maze
env.addObstacleLine(np.array([-5, -5]), np.array([5, -5]), 1)  # Bottom border
env.addObstacleLine(np.array([-5, 5]), np.array([5, 5]), 1)    # Top border
env.addObstacleLine(np.array([-5, -5]), np.array([-5, 5]), 1)  # Left border
env.addObstacleLine(np.array([5, -5]), np.array([5, 5]), 1)    # Right border

# Add internal walls to form the maze
env.addObstacleLine(np.array([-5, -3]), np.array([-2, -3]), 1)  # Horizontal lower left
env.addObstacleLine(np.array([2, -3]), np.array([5, -3]), 1)    # Horizontal lower right
env.addObstacleLine(np.array([-5, 3]), np.array([-2, 3]), 1)    # Horizontal upper left
env.addObstacleLine(np.array([2, 3]), np.array([5, 3]), 1)      # Horizontal upper right
env.addObstacleLine(np.array([-2, -3]), np.array([-2, 0]), 1)   # Vertical lower left
env.addObstacleLine(np.array([2, -3]), np.array([2, 0]), 1)     # Vertical lower right
env.addObstacleLine(np.array([-2, 3]), np.array([-2, 5]), 1)    # Vertical upper left
env.addObstacleLine(np.array([2, 3]), np.array([2, 5]), 1)      # Vertical upper right

# Set a starting position and a goal
env.setGoal(np.array([-4,-2]))
env.setPosition(np.array([4, -4, 0]))

# Adjust other parameters if needed
env.max_steps = 10

# You can proceed with simulation or creating movie as needed
# createMovie(new_net, 'new_simulation')
createMovie(randomnet,'randomnet')

Moviepy - Building video /content/videos/randomnet/rl-video-episode-0.mp4.
Moviepy - Writing video /content/videos/randomnet/rl-video-episode-0.mp4



Moviepy - Done !
Moviepy - video ready /content/videos/randomnet/rl-video-episode-0.mp4


Moviepy - Building video /content/videos/randomnet/rl-video-episode-0.mp4.
Moviepy - Writing video /content/videos/randomnet/rl-video-episode-0.mp4



Moviepy - Done !
Moviepy - video ready /content/videos/randomnet/rl-video-episode-0.mp4


In [ ]:
import copy


#hyper-parameters
TotalEpisodes=20000;
MaxSteps=5000;
ControlSteps=1; # number of iterations the same action is executed in the environment
FrameStack=1 # number of consecutive frames used to represent the state
FreezeCounter=25; # clone the model every X episodes
BatchSize=512;
exploration_threshold=1
exploration_threshold_min=0.01
exploration_decay=0.0013/16
discount_factor=0.9999
LearningRate=0.0001
BufferSize = 20000
env.max_steps=MaxSteps

#network DQN
policy_net = DQN((3+11*11)*FrameStack, n_actions).to(device)
target_net = DQN((3+11*11)*FrameStack, n_actions).to(device)
target_net.load_state_dict(policy_net.state_dict())
target_net.eval()

buffer = ReplayBuffer(BufferSize);
optimizer = torch.optim.Adam(policy_net.parameters(), lr=LearningRate)
loss=  torch.nn.SmoothL1Loss()

loss_val,scores, episodes,events, avg_scores,avg_scores20,exploration = [],[],[], [],[], [], []

bestScore=-99999;
at=0;
bestNet=copy.deepcopy(policy_net);


for f in range(TotalEpisodes):
    done  = False
    score = 0.0
    tloss = 0.0

    new_state,_=env.reset()

    if(env.mode==Modes.PoseAndLocalMapInline):
      stackedStateX=[new_state for i in range(FrameStack)];
      state=torch.from_numpy(np.reshape(np.array(stackedStateX),(FrameStack,3+env.local_grid_size*env.local_grid_size,1))).float().to(device)

    if(env.mode==Modes.PoseOnly ):
      stackedStateX=[new_state for i in range(FrameStack)];
      state=torch.from_numpy(np.reshape(np.array(stackedStateX),(FrameStack,3,1))).float().to(device)

    if(env.mode==Modes.PoseAndLocalMap):
      stackedStateX=[np.rollaxis(new_state[1], 2, 0).copy() for i in range(FrameStack)];
      state=torch.from_numpy(np.reshape(np.array(stackedStateX),(FrameStack,env.local_grid_size,env.local_grid_size))).float().to(device)
      stackedStatepose=[new_state[0] for i in range(FrameStack)];
      statepose=torch.from_numpy(np.reshape(np.array(stackedStatepose),(FrameStack,3,1))).float().to(device)

    if(env.mode==Modes.GlobalMap):
      stackedStateX=[np.rollaxis(new_state, 2, 0).copy() for i in range(FrameStack)];
      state=torch.from_numpy(np.reshape(np.array(stackedStateX),(FrameStack,env.grid_size,env.grid_size))).float().to(device)


    if f % FreezeCounter == 0:
       print("########################################"+str(f)+" of "+str(TotalEpisodes), exploration_threshold)
       target_net.load_state_dict(policy_net.state_dict())

    resetStatus=0;
    i=0

    for F in range(MaxSteps):
        action = policy_net.getPolicy(state,exploration_threshold)
        reward=0
        for _ in range(ControlSteps):
            new_state, r, done,trunc, info = env.step(action.item())
            reward+=r
            i=i+1
            if(done or trunc):
                break

        oldstate=state.clone();
        stackedStateX.pop(0)

        if(env.mode==Modes.PoseAndLocalMapInline):
          stackedStateX.append(new_state.copy())
          state=torch.from_numpy(np.reshape(np.array(stackedStateX),(FrameStack,3+env.local_grid_size*env.local_grid_size,1))).float().to(device)

        if(env.mode==Modes.PoseOnly):
          stackedStateX.append(new_state.copy())
          state=torch.from_numpy(np.reshape(np.array(stackedStateX),(FrameStack,3,1))).float().to(device)

        if(env.mode==Modes.PoseAndLocalMap):
          stackedStateX.append(np.rollaxis(new_state[1], 2, 0).copy())
          state=torch.from_numpy(np.reshape(np.array(stackedStateX),(FrameStack,env.local_grid_size,env.local_grid_size))).float().to(device)
          stackedStatepose.pop(0)
          stackedStatepose.append(new_state[0].copy())
          statepose=torch.from_numpy(np.reshape(np.array(stackedStatepose),(FrameStack,3,1))).float().to(device)

        if(env.mode==Modes.GlobalMap):
          stackedStateX.append(np.rollaxis(new_state, 2, 0).copy())
          state=torch.from_numpy(np.reshape(np.array(stackedStateX),(FrameStack,env.grid_size,env.grid_size))).float().to(device)


        score += reward
        if(F<(MaxSteps-1)):
            buffer.store_tuples(oldstate, action, torch.tensor(reward), state, torch.tensor(int(done)))

        trainModel(buffer,policy_net,target_net,optimizer,loss,BatchSize,discount_factor,False)


        if(done or trunc):
            break
    exploration_threshold= exploration_threshold-exploration_decay if exploration_threshold > exploration_threshold_min else exploration_threshold_min
    print("EP: ",score," It: ",F," eth: ",exploration_threshold)

    if(score>bestScore and exploration_threshold<0.1):
        print("Best EP: ",score," It: ",F," eth: ",exploration_threshold)
        bestScore=score;
        torch.save(policy_net.state_dict(), 'BestRobot_model.ckpt')
        at=f;

    exploration.append(exploration_threshold)
    scores.append(score)
    episodes.append(f)
    events.append(F)
    avg_scores.append(score/F)
    avg_scores20.append(np.mean(scores[-20:]))




In [ ]:
import matplotlib
matplotlib.use('module://matplotlib_inline.backend_inline')
plt.figure(figsize=(12, 6), dpi=80)
plt.plot(episodes, scores)
plt.plot(episodes, events)
plt.plot(episodes, avg_scores)
plt.plot(episodes, avg_scores20)
plt.plot(episodes, exploration)
plt.xlabel('episodes')
plt.ylabel('y axis label')

plt.title('Track Locked DQN '+str(usedoubleDQN)+' '+str(useDueling))
plt.legend(['scores', 'events', 'avg_scores', 'avg_scores100','exploration'])

plt.show()

In [ ]:
policy_net.load_state_dict(torch.load('BestRobot_model.ckpt'))
createMovie(policy_net,'bestNet')